In [4]:
import numpy as np
import numpy.linalg as LA
from scipy import integrate

## Метод колокації для інтегрального рівняння Фредгольма другого роду

Маємо інтегральне рівняння вигляду

$$
\varphi(x)-\lambda \int_{a}^{b} \varphi(y) K(x, y) d y = f(x), \quad x \in[a, b]
$$

Наближений розв'язок будемо шукати у вигляді

$$
\tilde{\varphi}(x)=\sum_{j=1}^{n} c_{j} \gamma_{j}(x)
$$

де $c_j$ - невідомі константи, які потрібно знайти, а $\gamma_j$ - задані лінійнонезалежні функції (в нашому випадку кусково аналітичні), $j = 1, ..., n$.

Тоді отримаємо такий вираз

$$
\sum_{j=1}^{n} c_{j}\left[\gamma_{j}(x)-\lambda \int_{a}^{b} K(x, y) \gamma_{j}(y) d y\right]=f(x)
$$

Розглянувши його на множині точок $x_i$, таких, що
$$
a \leqslant x_{1}<x_{2}<\ldots<x_{m} \leqslant b
$$

отримаємо СЛАР відносно невідомих $с_j$, $j = 1, ..., n$. 

In [91]:
a, b = -1, 1
K = lambda x, y: x*y + x**2
f = lambda x: 1
lambd = 1
h = 0.2

In [92]:
xx = np.arange(a, b + 0.000001, h).round(3).tolist()
n = len(xx)
A = np.zeros((n,n)).tolist()
xx

[-1.0, -0.8, -0.6, -0.4, -0.2, -0.0, 0.2, 0.4, 0.6, 0.8, 1.0]

In [93]:
def basicFunction(j, x, xx=xx, h=h):
  '''j=0...n-1, базис на [xx[j], xx[j+1]]'''
  n = len(xx)
  #P0 = 1
  
  #P1 = lambda y: (y - xx[j])/h

  #print(x)
  
  if j >=1 and xx[j-1] <= x <= xx[j]:
    return (x - xx[j-1])/h
  elif j <= n-1 and xx[j] <= x <= xx[j+1]:
    return (xx[j+1] - x)/h
  else:
    return 0
  #return x
  
  
  #for x_ in range(n):
    #1 + (x - xx[j])/h

In [94]:
for i in range(n):
  for j in range(n):
    
    Kxi_li= lambda s: K(xx[i], s) * basicFunction(j, s)
    
    (integral, err) = integrate.quad(Kxi_li, a, b)
    
    #print(a, b, integral, basicFunction(j, xx[i]))
    
    A[i][j] = basicFunction(j, xx[i]) - integral
    
    #integral = sm.integrate(K(xx[i], y)*, (y, a, b))
    #A[i,j] = (basicFunction(j, x) - integral).evalf(5,subs={x: xx[i]})

np.round(A, 3)

array([[ 0.807, -0.36 , -0.32 , -0.28 , -0.24 , -0.2  , -0.16 , -0.12 ,
        -0.08 , -0.04 , -0.007],
       [-0.139,  0.744, -0.224, -0.192, -0.16 , -0.128, -0.096, -0.064,
        -0.032,  0.   ,  0.011],
       [-0.092, -0.168,  0.856, -0.12 , -0.096, -0.072, -0.048, -0.024,
        -0.   ,  0.024,  0.02 ],
       [-0.053, -0.096, -0.08 ,  0.936, -0.048, -0.032, -0.016, -0.   ,
         0.016,  0.032,  0.021],
       [-0.023, -0.04 , -0.032, -0.024,  0.984, -0.008, -0.   ,  0.008,
         0.016,  0.024,  0.015],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  1.   ,  0.   ,  0.   ,
         0.   ,  0.   ,  0.   ],
       [ 0.015,  0.024,  0.016,  0.008, -0.   , -0.008,  0.984, -0.024,
        -0.032, -0.04 , -0.023],
       [ 0.021,  0.032,  0.016, -0.   , -0.016, -0.032, -0.048,  0.936,
        -0.08 , -0.096, -0.053],
       [ 0.02 ,  0.024, -0.   , -0.024, -0.048, -0.072, -0.096, -0.12 ,
         0.856, -0.168, -0.092],
       [ 0.011,  0.   , -0.032, -0.064, -0.096, -0.128,

In [95]:
ff = [[f(xx[j])] for j in range(n)]
ff

[[1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1]]

In [96]:
A = np.array(A, dtype='float')
np.round(A, 3)

array([[ 0.807, -0.36 , -0.32 , -0.28 , -0.24 , -0.2  , -0.16 , -0.12 ,
        -0.08 , -0.04 , -0.007],
       [-0.139,  0.744, -0.224, -0.192, -0.16 , -0.128, -0.096, -0.064,
        -0.032,  0.   ,  0.011],
       [-0.092, -0.168,  0.856, -0.12 , -0.096, -0.072, -0.048, -0.024,
        -0.   ,  0.024,  0.02 ],
       [-0.053, -0.096, -0.08 ,  0.936, -0.048, -0.032, -0.016, -0.   ,
         0.016,  0.032,  0.021],
       [-0.023, -0.04 , -0.032, -0.024,  0.984, -0.008, -0.   ,  0.008,
         0.016,  0.024,  0.015],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  1.   ,  0.   ,  0.   ,
         0.   ,  0.   ,  0.   ],
       [ 0.015,  0.024,  0.016,  0.008, -0.   , -0.008,  0.984, -0.024,
        -0.032, -0.04 , -0.023],
       [ 0.021,  0.032,  0.016, -0.   , -0.016, -0.032, -0.048,  0.936,
        -0.08 , -0.096, -0.053],
       [ 0.02 ,  0.024, -0.   , -0.024, -0.048, -0.072, -0.096, -0.12 ,
         0.856, -0.168, -0.092],
       [ 0.011,  0.   , -0.032, -0.064, -0.096, -0.128,

In [97]:
ff = np.array(ff, dtype='float')
ff.shape

(11, 1)

In [98]:
LA.det(A)

0.10666666664927221

In [99]:
c = LA.solve(A, ff)
c

array([[7.25],
       [5.  ],
       [3.25],
       [2.  ],
       [1.25],
       [1.  ],
       [1.25],
       [2.  ],
       [3.25],
       [5.  ],
       [7.25]])

In [ ]:
c.shape

In [ ]:
c.round(3)

In [ ]:
np.dot(A, c)

In [ ]:
y_f = 
for i in range(n):
  y_f += c[i][0]*basicFunction(i, y)
  
y_f

In [ ]:
6*x**2 + 1